In [3]:
import pandas as pd

from Bio import PDB

# 3->1-betűs aminosav-kód konverzió dictionary
from Bio.Data.IUPACData import protein_letters_3to1

In [3]:
basedir='C:\\Users\\David\\Documents\\sh2db\\'

In [108]:
pfam = pd.read_csv(basedir+'data/SH2_domain_containing_prot_right_resnum_with_pdb_nums_domcol_0503.csv')
pfam

,Unnamed: 0,UniProt entry,Gene_domain,Pole,UniProt residues,PDB ID,PDB chain ID,PDB residues,Category,Gene name,...,Res_stop,Uniprot_ID,New_uniprot_start,New_uniprot_stop,Old_uniprot_start,Old_uniprot_stop,diff_start,New_PDB_start,diff_stop,New_PDB_stop
0,0,3BP2_HUMAN,SH3BP2,N,458 - 538,2CR4,A,20 - 100,SigReg,SH3BP2,...,100,P78314,435,561,458,538,23,0,23,123
1,1,ABL1_HUMAN,ABL1,N,127 - 202,1AB2,A,11 - 86,Kinases,ABL1,...,86,P00519,105,228,127,202,22,0,26,112
2,2,ABL1_HUMAN,ABL1,N,127 - 202,1OPL,A,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247
3,3,ABL1_HUMAN,ABL1,N,127 - 202,1OPL,B,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247
4,4,ABL1_HUMAN,ABL1,N,127 - 202,2ABL,A,146 - 221,Kinases,ABL1,...,221,P00519,105,228,127,202,22,124,26,247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,637,ZAP70_HUMAN,ZAP70_C,C,164 - 239,2OQ1,A,165 - 241,Kinases,ZAP70,...,241,P43403,650,772,164,239,-486,651,533,774
637,638,ZAP70_HUMAN,ZAP70_C,C,165 - 239,2OZO,A,163 - 239,Kinases,ZAP70,...,239,P43403,650,772,165,239,-485,648,533,772
638,639,ZAP70_HUMAN,ZAP70_C,C,166 - 239,4K2R,A,163 - 239,Kinases,ZAP70,...,239,P43403,650,772,166,239,-484,647,533,772
639,640,ZAP70_HUMAN,ZAP70_C,C,167 - 239,4XZ0,A,163 - 239,Kinases,ZAP70,...,239,P43403,650,772,167,239,-483,646,533,772


In [161]:
table = pd.read_excel(basedir+'data/table_from_alignment_without_pdbs.xlsx', index_col = [0,1], header=None)
table

2   3   4   5    6    7    8    9    10   11   ...  241  242  243  \
0      1                                                   ...                  
STAT5A seq    -   -   -   -    -    -    N    Y    T    F  ...    A    D    A   
       nums   -   -   -   -    -    -  567  568  569  570  ...  711  712  713   
STAT5B seq    -   -   -   -    G    R    N    Y    T    F  ...    A    D    A   
       nums                  565  566  567  568  569  570  ...  716  717  718   
STAT6  seq    -   -   -   -    -    -    -    -    -    -  ...    -    -    -   
...          ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
TEC    nums   -   -   -   -    -    -    -    -    -  225  ...    -    -    -   
TXK    seq    -   -   -   -    -    -    -    -    -    N  ...    -    -    -   
       nums                                           128  ...    -    -    -   
BMX    seq    -   -   -   -    -    -    -    -    -    K  ...    -    -    -   
       nums                                           274  ...    -    -    -   

             244  245  246  247  248  249  250  
0      1                                        
STAT5A seq     G    G    S    S    A    T    Y  
       nums  714  715  716  717  718  719  720  
STAT5B seq     G    G    G    S    A    T    Y  
       nums  719  720  721  722  723  724  725  
STAT6  seq     -    -    -    -    -    -    -  
...          ...  ...  ...  ...  ...  ...  ...  
TEC    nums    -    -    -    -    -    -    -  
TXK    seq     -    -    -    -    -    -    -  
       nums    -    -    -    -    -    -    -  
BMX    seq     -    -    -    -    -    -    -  
       nums    -    -    -    -    -    -    -  

[240 rows x 249 columns]

In [162]:
parser = PDB.PDBParser(QUIET=True)

failcounter2 = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
                    pdb_id = str(pdb+'_'+chain+'_SH2_'+pole)
                    try:
                        #print(pdb_id)
                        structure = parser.get_structure(pdb_id, basedir+'Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_1_'+chain+'_SH2_'+pole+'.pdb') # struct delles pdb-fájlokat nem szereti!
                    except ValueError:
                        print('No structure for ',cat,gene,pdb)
                        continue
                    #model = structure[0]
                    residues = PDB.Selection.unfold_entities(structure, "R")
                    chain = pdb_id.split('_')[1]
                    #print("RESIDUES", residues)
                    #for residue in residues:
                    #    if residue.get_resname() in ['Hoh','HOH']: #This can be expanded if multiple problematic residues occur
                    #        residues.remove(residue) # dropping key in the next line (for "Hoh")
                    #print(residues)
                    
                    resnames = []
                    resids = []
                    
                    for i in residues:
                        if i.get_resname().title() in protein_letters_3to1:
                            resnames.append(protein_letters_3to1[i.get_resname().title()])                         
                            resids.append(i.get_id()[1])
                        elif i.get_resname().title() not in ['Hoh']:
                            resnames.append(i.get_resname().title())
                            resids.append(i.get_id()[1])
                    
                    pdbseq=list(zip(resnames,resids))
                    #try:
                    #    pdbseq = list(zip([ protein_letters_3to1[i.get_resname().title()] for i in residues],
                    #                      [i.get_id()[1] for i in residues]))
                    #except KeyError: #Hoh caused the Keyerror problem but was hard to remove
                    #    print('PDBSEQ failed',pdb,residues)
                    #    continue
                    #print(cat,gene,pdb,chain,pole)#, pdbseq)
                    increment1 = pfam[(pfam['PDB ID']==pdb) & (pfam["Pole"] == pole) & (pfam['PDB chain ID']==chain)]['New_uniprot_stop'].astype(int) - \
                                pfam[(pfam['PDB ID']==pdb) & (pfam["Pole"] == pole) & (pfam['PDB chain ID']==chain)]['New_PDB_stop'].astype(int)
                    increment2 = pfam[(pfam['PDB ID']==pdb) & (pfam["Pole"] == pole) & (pfam['PDB chain ID']==chain)]['New_uniprot_start'].astype(int) - \
                                pfam[(pfam['PDB ID']==pdb) & (pfam["Pole"] == pole) & (pfam['PDB chain ID']==chain)]['New_PDB_start'].astype(int)
                    increment = max(increment1.values[0],increment2.values[0])
                    # <- idáig jó - inentől nem -->
                    #print("INCREMENT ", increment)
                    failcounter1=0
                    for (resname,resid) in pdbseq:
                        try:
                            if gene in table.index.levels[0]:
                                #s=table.loc[(gene ,'nums')]==str(resid+increment.values[0])
                                s=table.loc[(gene ,'nums')]==resid+increment#.values[0] #korabban sztringkent kerestuk, most ugy nez ki, nem kell atalakitani
                                #print("S", s)
                                col=s[s].index[0]
                                #print("COL", col)
                                #print("GENE", gene)
                                #print("PDB ID", pdb_id)
                                table.loc[(gene, pdb_id),col]=resname
                            else:
                                s=table.loc[(gene+'_'+pole,'nums')]==resid+increment
                                col=s[s].index[0]
                                table.loc[(gene+'_'+pole, pdb_id),col]=resname

                        except (KeyError,IndexError) as e:
                            failcounter1 += 1
                            continue
                    if failcounter1 > 50:
                        print('FAILED', pdbseq)
                        failcounter2 += 1

print('Failed for '+str(failcounter2)+' entries')

newindex=table.index.sortlevel()[0]                           
table=table.reindex(index=newindex) # Összerendezzük az új sorokat a megfelelő fehérje entry-hez
table.to_excel(basedir+'data/table_from_alignment_with_pdbs_proba9.xlsx')

No structure for  Kinases ZAP70 1M61
No structure for  Kinases ZAP70 2OQ1
No structure for  Kinases ZAP70 4XZ0
No structure for  Kinases ZAP70 4XZ1
Failed for 0 entries


In [140]:
cat,gene,pdb,chain,pole

('Kinases', 'SYK', '1A81', 'A', 'N')

In [143]:
gene in table.index.levels[0]

False

In [5]:
pd.__version__

'1.1.3'

In [ ]:
# EZ NEM FOG KELLENI, A FENTIVEL DOLGOZTAM

parser = PDB.PDBParser(QUIET=True)
x = 0

for cat in pfam["Category"].unique():
    for gene in pfam[pfam["Category"] == cat]["Gene name"].unique():
        for pdb in pfam[pfam["Gene name"] == gene]["PDB ID"].unique():
            for chain in pfam[pfam["PDB ID"] == pdb]["PDB chain ID"].unique():
                for pole in pfam[(pfam["PDB ID"] == pdb) & (pfam["PDB chain ID"] == chain)]["Pole"].unique():
                    pdb_id = str(pdb+'_'+chain+'_SH2'+pole)                    
                    try:
                        #print(pdb_id)
                        structure = parser.get_structure(pdb_id, basedir+'Structures/'+cat+'/'+gene+'/'+pdb+'/'+pdb+'_'+chain+'_SH2_'+pole+'.pdb') # struct delles pdb-fájlokat nem szereti!
                    except ValueError:
                        continue
                    #model = structure[0]
                    residues = PDB.Selection.unfold_entities(structure, "R")
                    chain = (pdb+'_'+chain+'_SH2'+pole).split('_')[1]
                    """for residue in residues:
                        if residue in ['Hoh']: #This can be expanded if multiple problematic residues occur
                            residues.remove(residue) # dropping key in the next line (for "Hoh")
                    print(residues)"""
                    try:
                        pdbseq = list(zip([protein_letters_3to1[i.get_resname().title()] for i in residues],[i.get_id()[1] for i in residues]))
                    except KeyError: #Hoh caused the Keyerror problem but was hard to remove
                        continue
                    if x % 1 == 0:
                        print("PDB ID", pdb_id, "\n", "CHAIM", chain, "\n", "PDBSEQ", pdbseq)